# Deliverable 7.1: Nonlinear MPC for Landing

Implement and test a nonlinear MPC controller using CasADi for the full 12-state rocket system.

In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
from src.rocket import Rocket
from src.pos_rocket_vis import *
from LandMPC.nmpc_land import NmpcCtrl
import numpy as np

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7  # Do not change!!!
rocket.controller_type = 'NmpcCtrl'  # IMPORTANT: Set for proper NMPC constraint checking

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
# Landing maneuver: from (3, 2, 10, 30°) to (1, 0, 3, 0°)
sim_time = 15  # simulation length in seconds

# Initial state: at (3, 2, 10, 30°)
x0 = np.array([0, 0, 0,  # angular velocities (wx, wy, wz)
               0, 0, np.deg2rad(30),  # angles (alpha, beta, gamma/roll)
               0, 0, 0,  # linear velocities (vx, vy, vz)
               3, 2, 10])  # positions (x, y, z)

# Target state: (1, 0, 3, 0°)
x_ref = np.array([0.]*9 + [1., 0., 3.])  # Target: x=1, y=0, z=3

# NMPC parameters - use longer horizon for large maneuvers
H = 4.0  # horizon time (increased from 2.0 for better long-term planning)

# Create nonlinear MPC controller (it will trim around x_ref internally)
nmpc = NmpcCtrl(rocket, Ts=Ts, H=H, x_ref=x_ref)
print("\nNonlinear MPC initialized")
print(f"  - Horizon: {nmpc.N} steps ({H}s)")
print(f"  - State dimension: {nmpc.nx}")
print(f"  - Input dimension: {nmpc.nu}")
print(f"  - Uses full nonlinear dynamics (no subsystem decomposition)")
print(f"  - Integration: RK4 (more accurate than Euler)")
print(f"  - Tuning: High terminal cost (P=10*Q) for convergence")

# Store trim point for later use
xs, us = nmpc.xs, nmpc.us

## Test 1: Check Open-Loop Trajectory

Before running closed-loop, verify that the optimal open-loop trajectory from the initial state is reasonable.

In [ ]:
# Compute open-loop optimal trajectory
u0, x_ol, u_ol, t_ol = nmpc.get_u(0.0, x0)

print("Open-loop trajectory computed:")
print(f"  - Initial control: {u0}")
print(f"  - Final state: {x_ol[:, -1]}")
print(f"  - Target state: {xs}")

# Plot open-loop trajectory
import matplotlib.pyplot as plt
fig, axs = plt.subplots(3, 1, figsize=(10, 8))

# Positions
axs[0].plot(t_ol, x_ol[9, :], label='x')
axs[0].plot(t_ol, x_ol[10, :], label='y')
axs[0].plot(t_ol, x_ol[11, :], label='z')
axs[0].axhline(y=xs[9], color='r', linestyle='--', label='x_target')
axs[0].axhline(y=xs[10], color='g', linestyle='--', label='y_target')
axs[0].axhline(y=xs[11], color='b', linestyle='--', label='z_target')
axs[0].set_ylabel('Position [m]')
axs[0].legend()
axs[0].grid(True)

# Angles
axs[1].plot(t_ol, np.rad2deg(x_ol[3, :]), label='alpha')
axs[1].plot(t_ol, np.rad2deg(x_ol[4, :]), label='beta')
axs[1].plot(t_ol, np.rad2deg(x_ol[5, :]), label='gamma (roll)')
axs[1].set_ylabel('Angles [deg]')
axs[1].legend()
axs[1].grid(True)

# Inputs
axs[2].plot(t_ol[:-1], u_ol[0, :], label='d1')
axs[2].plot(t_ol[:-1], u_ol[1, :], label='d2')
axs[2].plot(t_ol[:-1], u_ol[2, :], label='Pavg')
axs[2].plot(t_ol[:-1], u_ol[3, :], label='Pdiff')
axs[2].set_xlabel('Time [s]')
axs[2].set_ylabel('Inputs')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

## Test 2: Closed-Loop Simulation

In [ ]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(nmpc, sim_time, H, x0)

## Diagnostics: Check NMPC Setup

Before simulating, let's verify the NMPC controller is working correctly.

In [ ]:
# Test 1: Check if NMPC solver is working
print("Testing NMPC get_u() with initial state...")
try:
    u_test, x_test, u_test_traj, t_test = nmpc.get_u(0.0, x0)
    print(f"✓ NMPC solver succeeded!")
    print(f"  Control output: {u_test}")
    print(f"  Expected hover: {us}")
    print(f"  Difference: {u_test - us}")
    print(f"  Trajectory shape: states={x_test.shape}, inputs={u_test_traj.shape}")
    
    # Check if control is reasonable
    if np.allclose(u_test, [0, 0, 0, 0], atol=1.0):
        print(f"\n⚠️ WARNING: Control is nearly zero - NMPC might be returning default values")
    else:
        print(f"\n✓ Control seems reasonable (non-zero)")
        
except Exception as e:
    print(f"✗ NMPC solver FAILED with error:")
    print(f"  {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)

## Analysis

**Questions to answer:**
1. Does the NMPC successfully land the rocket at the target (1, 0, 3) with 0° roll?
2. How does NMPC performance compare with the linear MPC from Part 6?
3. Are all constraints satisfied (z ≥ 0, |β| ≤ 80°, input bounds)?
4. What are the advantages of NMPC over linear MPC for this task?
5. What is the computational cost (solve time per iteration)?

In [ ]:
# Compare final state with target
print("\nLanding Performance:")
print(f"  Target position: ({xs[9]:.2f}, {xs[10]:.2f}, {xs[11]:.2f})")
print(f"  Final position:  ({x_cl[9, -1]:.2f}, {x_cl[10, -1]:.2f}, {x_cl[11, -1]:.2f})")
print(f"  Position error:  ({abs(x_cl[9, -1] - xs[9]):.3f}, {abs(x_cl[10, -1] - xs[10]):.3f}, {abs(x_cl[11, -1] - xs[11]):.3f})")
print(f"\n  Target roll: {np.rad2deg(xs[5]):.2f}°")
print(f"  Final roll:  {np.rad2deg(x_cl[5, -1]):.2f}°")
print(f"  Roll error:  {np.rad2deg(abs(x_cl[5, -1] - xs[5])):.3f}°")

## Settling Time Verification

**Requirement:** Settling time ≤ 4 seconds when landing from (3, 2, 10, 30°) to (1, 0, 3, 0°)

In [ ]:
# Compute settling time (when all states are within tolerance of target)
# Define tolerance: 2% for positions, 1° for angles, small for velocities
pos_tol = 0.02  # 2 cm for positions
vel_tol = 0.05  # 5 cm/s for velocities
angle_tol = np.deg2rad(1)  # 1 degree
ang_vel_tol = np.deg2rad(2)  # 2 deg/s

# Compute errors over time
pos_error = np.linalg.norm(x_cl[9:12, :] - xs[9:12, np.newaxis], axis=0)
vel_error = np.linalg.norm(x_cl[6:9, :] - xs[6:9, np.newaxis], axis=0)
angle_error = np.abs(x_cl[5, :] - xs[5])  # Roll angle (most critical)
ang_vel_error = np.linalg.norm(x_cl[0:3, :] - xs[0:3, np.newaxis], axis=0)

# Find when all errors are within tolerance
settled = (pos_error < pos_tol) & (vel_error < vel_tol) & \
          (angle_error < angle_tol) & (ang_vel_error < ang_vel_tol)

settled_indices = np.where(settled)[0]
if len(settled_indices) > 0:
    settling_time = t_cl[settled_indices[0]]
    print(f"✓ Settling time: {settling_time:.2f} s")
    print(f"  Requirement: ≤ 4.0 s")
    if settling_time <= 4.0:
        print(f"  STATUS: PASSED ✓")
    else:
        print(f"  STATUS: FAILED ✗")
else:
    settling_time = sim_time
    print(f"✗ System did not settle within {sim_time}s")
    print(f"  STATUS: FAILED ✗")

# Plot convergence
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

axs[0, 0].plot(t_cl, pos_error)
axs[0, 0].axhline(y=pos_tol, color='r', linestyle='--', label='Tolerance')
axs[0, 0].axvline(x=settling_time, color='g', linestyle='--', label=f'Settling time: {settling_time:.2f}s')
axs[0, 0].set_xlabel('Time [s]')
axs[0, 0].set_ylabel('Position Error [m]')
axs[0, 0].set_title('Position Convergence')
axs[0, 0].legend()
axs[0, 0].grid(True)

axs[0, 1].plot(t_cl, vel_error)
axs[0, 1].axhline(y=vel_tol, color='r', linestyle='--', label='Tolerance')
axs[0, 1].axvline(x=settling_time, color='g', linestyle='--', label=f'Settling time: {settling_time:.2f}s')
axs[0, 1].set_xlabel('Time [s]')
axs[0, 1].set_ylabel('Velocity Error [m/s]')
axs[0, 1].set_title('Velocity Convergence')
axs[0, 1].legend()
axs[0, 1].grid(True)

axs[1, 0].plot(t_cl, np.rad2deg(angle_error))
axs[1, 0].axhline(y=np.rad2deg(angle_tol), color='r', linestyle='--', label='Tolerance')
axs[1, 0].axvline(x=settling_time, color='g', linestyle='--', label=f'Settling time: {settling_time:.2f}s')
axs[1, 0].set_xlabel('Time [s]')
axs[1, 0].set_ylabel('Roll Error [deg]')
axs[1, 0].set_title('Roll Angle Convergence')
axs[1, 0].legend()
axs[1, 0].grid(True)

axs[1, 1].plot(t_cl, np.rad2deg(ang_vel_error))
axs[1, 1].axhline(y=np.rad2deg(ang_vel_tol), color='r', linestyle='--', label='Tolerance')
axs[1, 1].axvline(x=settling_time, color='g', linestyle='--', label=f'Settling time: {settling_time:.2f}s')
axs[1, 1].set_xlabel('Time [s]')
axs[1, 1].set_ylabel('Angular Velocity Error [deg/s]')
axs[1, 1].set_title('Angular Velocity Convergence')
axs[1, 1].legend()
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()

## Constraint Verification

**Safety-Critical Constraints:**
1. Ground collision: z ≥ 0
2. Singularity avoidance: |β| ≤ 80°
3. Input bounds: δ1, δ2 ∈ [-0.26, 0.26] rad, Pavg ∈ [40, 80]%

In [ ]:
# Verify all constraints are satisfied throughout the trajectory

print("=" * 60)
print("CONSTRAINT VERIFICATION")
print("=" * 60)

# 1. Ground collision constraint: z ≥ 0
min_z = np.min(x_cl[11, :])
print(f"\n1. Ground Collision (z ≥ 0):")
print(f"   Minimum altitude: {min_z:.4f} m")
if min_z >= 0:
    print(f"   STATUS: SATISFIED ✓")
else:
    print(f"   STATUS: VIOLATED ✗ (crashed at z = {min_z:.4f}m)")

# 2. Beta singularity constraint: |β| ≤ 80°
max_beta = np.max(np.abs(x_cl[4, :]))
beta_limit = np.deg2rad(80)
print(f"\n2. Beta Singularity (|β| ≤ 80°):")
print(f"   Max |β|: {np.rad2deg(max_beta):.2f}°")
print(f"   Limit:   80.00°")
if max_beta <= beta_limit:
    print(f"   STATUS: SATISFIED ✓")
else:
    print(f"   STATUS: VIOLATED ✗")

# 3. Input constraints
# Servo angles: |δ1|, |δ2| ≤ 15° = 0.26 rad
max_d1 = np.max(np.abs(u_cl[0, :]))
max_d2 = np.max(np.abs(u_cl[1, :]))
servo_limit = 0.26

print(f"\n3. Servo Deflection Angles (|δ| ≤ 0.26 rad = 15°):")
print(f"   Max |δ1|: {max_d1:.4f} rad = {np.rad2deg(max_d1):.2f}°")
print(f"   Max |δ2|: {max_d2:.4f} rad = {np.rad2deg(max_d2):.2f}°")
if max_d1 <= servo_limit and max_d2 <= servo_limit:
    print(f"   STATUS: SATISFIED ✓")
else:
    print(f"   STATUS: VIOLATED ✗")

# Average throttle: 40 ≤ Pavg ≤ 80
min_Pavg = np.min(u_cl[2, :])
max_Pavg = np.max(u_cl[2, :])

print(f"\n4. Average Throttle (40 ≤ Pavg ≤ 80):")
print(f"   Min Pavg: {min_Pavg:.2f}%")
print(f"   Max Pavg: {max_Pavg:.2f}%")
if min_Pavg >= 40 and max_Pavg <= 80:
    print(f"   STATUS: SATISFIED ✓")
else:
    print(f"   STATUS: VIOLATED ✗")

# Differential throttle: Pdiff ∈ [-20, 20]
max_Pdiff = np.max(np.abs(u_cl[3, :]))

print(f"\n5. Differential Throttle (|Pdiff| ≤ 20):")
print(f"   Max |Pdiff|: {max_Pdiff:.2f}%")
if max_Pdiff <= 20:
    print(f"   STATUS: SATISFIED ✓")
else:
    print(f"   STATUS: VIOLATED ✗")

print("\n" + "=" * 60)

# Plot constraints over time
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# z constraint
axs[0, 0].plot(t_cl, x_cl[11, :], label='z (altitude)')
axs[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Ground (z=0)')
axs[0, 0].fill_between(t_cl, -1, 0, color='red', alpha=0.2, label='Forbidden (z<0)')
axs[0, 0].set_xlabel('Time [s]')
axs[0, 0].set_ylabel('Altitude z [m]')
axs[0, 0].set_title('Ground Collision Constraint')
axs[0, 0].legend()
axs[0, 0].grid(True)
axs[0, 0].set_ylim([-0.5, max(x_cl[11, :]) + 1])

# Beta constraint
axs[0, 1].plot(t_cl, np.rad2deg(x_cl[4, :]), label='β (pitch)')
axs[0, 1].axhline(y=80, color='r', linestyle='--', linewidth=2, label='Limit (+80°)')
axs[0, 1].axhline(y=-80, color='r', linestyle='--', linewidth=2, label='Limit (-80°)')
axs[0, 1].fill_between(t_cl, 80, 90, color='red', alpha=0.2, label='Forbidden')
axs[0, 1].fill_between(t_cl, -90, -80, color='red', alpha=0.2)
axs[0, 1].set_xlabel('Time [s]')
axs[0, 1].set_ylabel('Beta β [deg]')
axs[0, 1].set_title('Beta Singularity Constraint')
axs[0, 1].legend()
axs[0, 1].grid(True)
axs[0, 1].set_ylim([-90, 90])

# Servo angles
axs[1, 0].plot(t_cl[:-1], np.rad2deg(u_cl[0, :]), label='δ1')
axs[1, 0].plot(t_cl[:-1], np.rad2deg(u_cl[1, :]), label='δ2')
axs[1, 0].axhline(y=15, color='r', linestyle='--', linewidth=2, label='Limit (±15°)')
axs[1, 0].axhline(y=-15, color='r', linestyle='--', linewidth=2)
axs[1, 0].fill_between(t_cl[:-1], 15, 20, color='red', alpha=0.2, label='Forbidden')
axs[1, 0].fill_between(t_cl[:-1], -20, -15, color='red', alpha=0.2)
axs[1, 0].set_xlabel('Time [s]')
axs[1, 0].set_ylabel('Servo Angles [deg]')
axs[1, 0].set_title('Servo Deflection Constraints')
axs[1, 0].legend()
axs[1, 0].grid(True)
axs[1, 0].set_ylim([-20, 20])

# Throttle
axs[1, 1].plot(t_cl[:-1], u_cl[2, :], label='Pavg', linewidth=2)
axs[1, 1].axhline(y=40, color='r', linestyle='--', linewidth=2, label='Min (40%)')
axs[1, 1].axhline(y=80, color='r', linestyle='--', linewidth=2, label='Max (80%)')
axs[1, 1].fill_between(t_cl[:-1], 0, 40, color='red', alpha=0.2, label='Forbidden')
axs[1, 1].fill_between(t_cl[:-1], 80, 100, color='red', alpha=0.2)
axs[1, 1].set_xlabel('Time [s]')
axs[1, 1].set_ylabel('Average Throttle [%]')
axs[1, 1].set_title('Throttle Constraints')
axs[1, 1].legend()
axs[1, 1].grid(True)
axs[1, 1].set_ylim([0, 100])

plt.tight_layout()
plt.show()

## Comparison: NMPC (Part 7.1) vs Linear MPC (Part 6.2)

**Task:** Compare NMPC controller vs "robust tube MPC (z-axis) + three nominal MPCs (x, y, roll)" from Deliverable 6.2

This section provides a qualitative comparison of the two approaches for the rocket landing task from (3, 2, 10, 30°) to (1, 0, 3, 0°).

In [ ]:
# Comparative Analysis Table
import pandas as pd

comparison_data = {
    'Aspect': [
        'Model Accuracy',
        'State Space Coverage',
        'Constraint Handling',
        'Coupling Effects',
        'Computational Cost',
        'Implementation Complexity',
        'Tuning Difficulty',
        'Settling Time (typical)',
        'Robustness (disturbances)',
        'Robustness (model mismatch)',
        'Real-time Feasibility',
        'Theoretical Guarantees'
    ],
    'Linear MPC (Part 6.2)': [
        'Approximate (linearization)',
        'Limited (near trim point)',
        'Tube MPC (z-axis only)',
        'Ignored (4 independent systems)',
        'Low (~1-5 ms/iteration)',
        'Moderate (4 controllers)',
        'Moderate (4 sets of params)',
        '~3-4 seconds',
        'Good (tube MPC for z)',
        'Poor (linearization errors)',
        'Excellent (fast)',
        'Strong (recursive feasibility)'
    ],
    'NMPC (Part 7.1)': [
        'Exact (full nonlinear)',
        'Global (entire state space)',
        'Direct (hard constraints)',
        'Captured (fully coupled)',
        'High (~50-200 ms/iteration)',
        'Lower (single controller)',
        'Easier (single cost)',
        f'~{settling_time:.1f} seconds',
        'Moderate (no explicit robustness)',
        'Excellent (no linearization)',
        'Challenging (slow solves)',
        'Weak (local optimality only)'
    ]
}

df_comparison = pd.DataFrame(comparison_data)

# Display as formatted table
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
print("\n" + "=" * 100)
print("COMPARATIVE ANALYSIS: NMPC vs LINEAR MPC")
print("=" * 100)
print(df_comparison.to_string(index=False))
print("=" * 100)

### Pros and Cons Summary

#### Linear MPC (Part 6.2) - Pros ✓
1. **Computational efficiency**: Fast enough for high-rate control (20 Hz feasible)
2. **Theoretical guarantees**: Recursive feasibility via terminal sets, proven stability
3. **Tube MPC robustness**: Explicit disturbance rejection for z-axis (most critical)
4. **Predictable behavior**: Linear predictions easier to debug and verify
5. **Real-time ready**: Deterministic solve times, suitable for embedded systems

#### Linear MPC (Part 6.2) - Cons ✗
1. **Linearization errors**: Poor performance far from trim point (large angles, high velocities)
2. **Coupling ignored**: Treating x, y, z, roll independently misses cross-axis dynamics
3. **Conservative constraints**: Must tighten constraints to account for linearization errors
4. **Limited maneuvers**: Cannot exploit full nonlinear capabilities of the system
5. **Multiple controllers**: Coordination overhead, tuning complexity

---

#### NMPC (Part 7.1) - Pros ✓
1. **Accuracy**: Exact dynamics, valid across entire operating range
2. **Unified controller**: Single optimization handles all axes and coupling
3. **Aggressive maneuvers**: Can exploit full nonlinear performance envelope
4. **No linearization**: Works for large angle changes, high velocities
5. **Simpler tuning**: Single cost function instead of four separate controllers

#### NMPC (Part 7.1) - Cons ✗
1. **Computational burden**: 10-50× slower than linear MPC (may require code generation)
2. **No stability guarantees**: Only local optimality, no recursive feasibility proof
3. **Solver reliability**: Nonconvex problems may fail to converge or hit iteration limits
4. **No explicit robustness**: Doesn't explicitly handle disturbances (unlike tube MPC)
5. **Tuning sensitivity**: Terminal cost/constraints more critical, harder to get right

### Recommendation: When to Use Each Approach?

**Use Linear MPC (Part 6.2) when:**
- Real-time performance is critical (embedded systems, high control rates)
- Operating near a known equilibrium/trim point
- Formal stability/safety guarantees are required (certification, safety-critical)
- Limited computational resources (microcontrollers)
- Robust performance against disturbances is essential (tube MPC)

**Use NMPC (Part 7.1) when:**
- Large maneuvers away from equilibrium (aggressive landing, acrobatics)
- Coupled dynamics are significant (can't be decomposed)
- Accuracy matters more than speed (trajectory optimization, offline planning)
- Computational resources available (modern processors, GPU acceleration)
- Nonlinear constraints or performance objectives

**Hybrid Approach (Best of Both):**
For production rocket landing systems (e.g., SpaceX), consider:
1. **NMPC for trajectory planning** (offline, generate reference)
2. **Linear MPC for tracking** (online, fast feedback around NMPC trajectory)
3. **Gain scheduling** (multiple linear controllers at different operating points)

This combines NMPC's global accuracy with linear MPC's computational efficiency and theoretical guarantees.

## Summary of NMPC Implementation

**Key Design Decisions:**

1. **Integration Method**: RK4 (4th-order Runge-Kutta) instead of Euler for better accuracy with nonlinear dynamics

2. **Constraints Implemented**:
   - Ground collision: z ≥ 0 (safety-critical)
   - Singularity avoidance: |β| ≤ 80° (numerical stability)
   - Servo limits: |δ1|, |δ2| ≤ 0.26 rad (±15°)
   - Throttle: 40% ≤ Pavg ≤ 80% (safety + structural limits)
   - Differential throttle: |Pdiff| ≤ 20%

3. **Cost Function Tuning**:
   ```
   Q_angular_vel = 1.0    (allow rotation)
   Q_angles = 20.0        (penalize tilting)
   Q_velocities = 10.0    (reach zero velocity)
   Q_positions = [50, 50, 100]  (z most critical for landing)
   R_inputs = 0.01        (allow aggressive control)
   P_terminal = 10*Q      (ensure final convergence)
   ```

4. **Horizon Selection**: H = 4.0s (80 steps @ 20Hz)
   - Longer horizon needed for large maneuvers (7m distance, 30° rotation)
   - Trade-off: Better performance vs computational cost

5. **Solver Options**: IPOPT with 500 max iterations, tolerance 1e-5, expanded NLP